pip install requests  

pip install lxml  
pip install html5lib

pip install beautifulsoup4      
pip install bs4    

pip install -U selenium   

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from pathlib import Path
import urllib.request

In [2]:
# url = 'https://iutbox.iut.ac.ir/index.php/s/MtzkTTDnHrfte3R?path=%2FVideo'
url = 'https://iutbox.iut.ac.ir/index.php/s/MtzkTTDnHrfte3R'

MyPATH = Path('../FDM2021')

In [3]:
def newPath(url, newDir):
    """generate newURL and DownloadURL based on public link and directory

        Parameters
        ----------
        url : str
            Base url
        newDir : str
            directory that want to join with base url
    """
    try:
        base_url, path = url.split('?path=')
    except:
        base_url = url.split('?path=')[0]
        path = ''  #'%2F'
    
    newPath = path + ( '' if newDir=='' else ('%2F' + newDir) )
    
    if newPath == '':
        newPath = '%2F'
    
    newURL = base_url + '?path=' + newPath
    DownloadURL = base_url + "/download" + '?path=' + newPath + "&files="
    
    return newURL, DownloadURL 

In [4]:
def recPath(url, args):
    for item in args:
        url = newPath(url, item)[0]
    return url

In [5]:
url , downlaod_link = newPath(url, '')
url , downlaod_link

('https://iutbox.iut.ac.ir/index.php/s/MtzkTTDnHrfte3R?path=%2F',
 'https://iutbox.iut.ac.ir/index.php/s/MtzkTTDnHrfte3R/download?path=%2F&files=')

## Static Websites
https://realpython.com/beautiful-soup-web-scraper-python/#dynamic-websites

In [6]:
# r = requests.get(url, allow_redirects=True)
# html_doc = r.text

## Dynamic Websites
https://realpython.com/modern-web-automation-with-python-and-selenium/

In [7]:
def initialize_webdriver(url):
    options = webdriver.ChromeOptions()
    # options.add_argument("--enable-javascript")
    options.headless = True

    driver = webdriver.Chrome('./chromedriver.exe', options=options) # geckodriver.exe
    driver.implicitly_wait(3)
    driver.get(url) 

    # this is just to ensure that the page is loaded
    time.sleep(5) 
    
    
    #### For Scroll infinite in page    
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    html_doc = driver.page_source

    driver.close()

    soup = BeautifulSoup(html_doc, 'html.parser')  # 'html5lib'
    # print(soup.prettify())
    
    return soup

In [8]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def SimplePrint(num, Type, name , size, modified_original ):
    color = bcolors.OKGREEN if Type==1 else bcolors.FAIL
    print(f"{color}{num:02d}) {name}" + " "*((7*10 -(len(name)+3) )) + f"({size})  \t({modified_original}){bcolors.ENDC}")
    
def TreePrint(lvl, Type, name ):
    color = bcolors.OKGREEN if Type==1 else bcolors.FAIL
    print("\t"*lvl + f"{color}{name}{bcolors.ENDC}")

In [9]:
def lsl(url=None):
    if url==None:
        print("Please pass URL")
        return
    
    soup = initialize_webdriver(url)
    
    result = soup.find_all('span', class_='nametext') # driver.find_elements_by_class_name('innernametext') #
    filesize = soup.find_all('td', class_='filesize')
    modified_data = soup.find_all('span', class_='modified live-relative-timestamp')

    for num, item in enumerate(result, start=0):
        innernametext =  item.find('span', class_='innernametext').text
        extension =  item.find('span', class_='extension')
        size = filesize[num].text
        modified_relative = modified_data[num].text
        modified_original = modified_data[num].get('data-original-title')

        if extension == None :
            DirName = innernametext
            SimplePrint(num, 1, DirName , size, modified_original)
        else :
            filename = innernametext + extension.text
            SimplePrint(num, 0, filename , size, modified_original) 

In [10]:
lsl( newPath(url, '')[0] )

00) Datasets                                                           (۳٫۷ MB)  	(۲۸ فوریه ۲۰۲۱ ۲۰:۵۸)
01) Extra                                                              (۴۸٫۶ MB)  	(۲۶ فوریه ۲۰۲۱ ۱۰:۵۲)
02) Ref                                                                (۴۲٫۵ MB)  	(۳۱ مه ۲۰۲۱ ۱۸:۳۰)
03) Video                                                              (۱٫۵ GB)  	(۳۱ مه ۲۰۲۱ ۱۹:۳۱)
04) FDM2021-Class-Rules-V12.pdf                                        (268 KB)  	(۱۲ فوریه ۲۰۲۱ ۲۲:۳۹)
05) FDM2021-Class-Rules-V14.pdf                                        (269 KB)  	(۲۳ فوریه ۲۰۲۱ ۱۰:۴۰)
06) FDM2021-Class-Rules-V15.pdf                                        (299 KB)  	(۳۰ آوریل ۲۰۲۱ ۰۵:۳۷)


In [11]:
lsl( newPath(url, 'Ref')[0] )

00) BookParts                                                          (864 KB)  	(۲۱ مه ۲۰۲۱ ۱۲:۳۵)
01) Notebook                                                           (16 KB)  	(۱۲ آوریل ۲۰۲۱ ۱۳:۱۴)
02) Slides                                                             (۴٫۹ MB)  	(۳۱ مه ۲۰۲۱ ۱۸:۲۹)
03) BK1-IntroDM.pdf                                                    (۲۶٫۴ MB)  	(۵ فوریه ۲۰۲۱ ۲۰:۰۰)
04) BK2-DSPR.pdf                                                       (۴٫۷ MB)  	(۷ دسامبر ۲۰۲۰ ۱۸:۱۴)
05) BK3-DKID2nd.pdf                                                    (۵٫۷ MB)  	(۱۲ نوامبر ۲۰۱۴ ۰۱:۵۲)


In [12]:
def tree(url=None, ShowFiles=0, lvl=0):
    if url==None:
        print("Please pass URL")
        return
    
    soup = initialize_webdriver(url)
    
    result = soup.find_all('span', class_='nametext') # driver.find_elements_by_class_name('innernametext') #
    filesize = soup.find_all('td', class_='filesize')
    modified_data = soup.find_all('span', class_='modified live-relative-timestamp')

    for num, item in enumerate(result, start=0):
        innernametext =  item.find('span', class_='innernametext').text
        extension =  item.find('span', class_='extension')
        size = filesize[num].text
        modified_relative = modified_data[num].text
        modified_original = modified_data[num].get('data-original-title')

        if extension == None :
            DirName = innernametext
            TreePrint(lvl, 1, DirName)
            tree( newPath(url, DirName)[0] , ShowFiles, lvl+1)
        elif ShowFiles :
            filename = innernametext + extension.text
            TreePrint(lvl, 0, filename) 

In [13]:
tree( newPath(url, '')[0], 1)

Datasets
	BK1-IntroDM-Datasets.zip
	BK2-DSPR-Datasets.zip
	BK3-DKID2e-Datasets.zip
Extra
	W02-Extra-Pandas-Samani-nr.mp4
Ref
	BookParts
		BK1-AssociationAnalysis-text.pdf
		Naive-Bayes-from-BK1-IntroDM-orig-marked.pdf
		NaiveBayes-from-Vishnu Pendyala - Veracity of Big Data.pdf
	Notebook
		NaiveBayes-and-Eval-BK2.Ch8and7.ipynb
	Slides
		BK1-Chapter2-data-sel1.pdf
		BK1-Chapter5_basic_association_analysis-sel.pdf
		BK1-Chapter6_advanced_association_analysis-sel.pdf
		BK1-Chapter7_basic_cluster_analysis-sel.pdf
		BK3-Chapter 1.pdf
		BK3-Chapter 2-sel.pdf
	BK1-IntroDM.pdf
	BK2-DSPR.pdf
	BK3-DKID2nd.pdf
Video
	W01-0-Course-Rules.mp4
	W01-1-DM-Intro-Process.mp4
	W02-1-DM-Intro-BK1.mp4
	W02-Extra-link-to-basics.mp4
	W03-1-part1-Data Preprocessing cont, (BK3.Ch2).mp4
	W03-1-part2-Problem Understanding (BK2.ch3.part1).mp4
	W03-1-part3-Data Preprocessing py (BK2.ch3.part2).mp4
	W04-1-part1-EDA (BK3.Ch3).mp4
	W04-1-part2-EDA (BK2.Ch4).mp4
	W05-1-Pre-Model (BK2.Ch5).mp4
	W05-2-Model-DeciTree-CART

In [14]:
def CheckExist(PathList, Name, lvl): 
    tmp = MyPATH
    for i in range(lvl+1):
        tmp = tmp.joinpath( PathList[i] )
    tmp = tmp.joinpath( Name ) 
    
    tmpFlag = tmp.exists()
    
    if tmpFlag :
        print("✅", tmp)
    else:
        print("❌", tmp)
        
    return tmpFlag

In [15]:
def FullCheck(url=None, PathList=[], ShowFiles=0, lvl=0):
    if url==None:
        print("Please pass URL")
        return
    
    url = recPath(url, PathList )
    
    soup = initialize_webdriver(url)
    
    result = soup.find_all('span', class_='nametext') # driver.find_elements_by_class_name('innernametext') #
    filesize = soup.find_all('td', class_='filesize')
    modified_data = soup.find_all('span', class_='modified live-relative-timestamp')

    for num, item in enumerate(result, start=0):
        innernametext =  item.find('span', class_='innernametext').text
        extension =  item.find('span', class_='extension')
        size = filesize[num].text
        modified_relative = modified_data[num].text
        modified_original = modified_data[num].get('data-original-title')

        if extension == None :
            DirName = innernametext
#             TreePrint(lvl, 1, DirName)
            
            if not CheckExist(PathList, DirName, lvl) :
                continue
                
            FullCheck( url , PathList+[DirName], ShowFiles, lvl+1)
        elif ShowFiles :
            filename = innernametext + extension.text
#             TreePrint(lvl, 0, filename) 
            
            if not CheckExist(PathList, filename, lvl) :
                continue
                 

In [16]:
FullCheck( url, [''], 1)

✅ ..\FDM2021\Datasets
✅ ..\FDM2021\Datasets\BK1-IntroDM-Datasets.zip
✅ ..\FDM2021\Datasets\BK2-DSPR-Datasets.zip
✅ ..\FDM2021\Datasets\BK3-DKID2e-Datasets.zip
✅ ..\FDM2021\Extra
✅ ..\FDM2021\Extra\W02-Extra-Pandas-Samani-nr.mp4
✅ ..\FDM2021\Ref
✅ ..\FDM2021\Ref\BookParts
✅ ..\FDM2021\Ref\Notebook
✅ ..\FDM2021\Ref\Slides
✅ ..\FDM2021\Ref\BK1-IntroDM.pdf
✅ ..\FDM2021\Ref\BK2-DSPR.pdf
✅ ..\FDM2021\Ref\BK3-DKID2nd.pdf
✅ ..\FDM2021\Video
✅ ..\FDM2021\Video\W01-0-Course-Rules.mp4
✅ ..\FDM2021\Video\W01-1-DM-Intro-Process.mp4
✅ ..\FDM2021\Video\W02-1-DM-Intro-BK1.mp4
❌ ..\FDM2021\Video\W02-Extra-link-to-basics.mp4
✅ ..\FDM2021\Video\W03-1-part1-Data Preprocessing cont, (BK3.Ch2).mp4
✅ ..\FDM2021\Video\W03-1-part2-Problem Understanding (BK2.ch3.part1).mp4
✅ ..\FDM2021\Video\W03-1-part3-Data Preprocessing py (BK2.ch3.part2).mp4
✅ ..\FDM2021\Video\W04-1-part1-EDA (BK3.Ch3).mp4
✅ ..\FDM2021\Video\W04-1-part2-EDA (BK2.Ch4).mp4
✅ ..\FDM2021\Video\W05-1-Pre-Model (BK2.Ch5).mp4
✅ ..\FDM2021\Video\W05

In [17]:
# urllib.request.urlretrieve(download_list[31], filename_list[31]) 